In [332]:
import os
import json
import glob
import numpy as np
import pandas as pd

In [ ]:
# Apache Airflow integrated into sagemaker to trigger workflow
#Bodo3 python script 

- call bash from py
- put in back in s3 using bodo3
- airflow triggg sagemaker
- google Airflow 1 on 1

In [29]:
path_to_json = "Dataset/json/"
json_files = [json for json in os.listdir(path_to_json)]
print(json_files)

['2019-08-20-traffic-analysis-exercise', 'dhcp.json', 'firefox58.json', 'ikev1.json', 'ikev2', 'kali-normal-ssh.json', 'kali-password-attack_hydra.json', 'malware_infection', 'openssl102n', 'sample', 'sample_tls12_handshake_0', 'snort_log1', 'ssh-dropbear-default', 'ssh-gex-sha256', 'tls10', 'tls11', 'tls12']


In [192]:
'''
Load files
'''


path_to_json = "Dataset/json/"
contents = []
pattern = os.path.join(path_to_json, '*')
file_list = glob.glob(pattern)
c = 0 
for file in file_list:
    try:
        with open(file, 'r', encoding='cp1252') as f:
            next(f)
            for line in f:
                try:
                    contents.append(json.loads(line))
                except:
                    c+=1
                    pass
    except UnicodeDecodeError as e:
        file_.append(file)
        pass

156


'\nKey: no value\n\n'

In [195]:
'''
Create dataframe
'''
df = pd.DataFrame.from_dict(contents)

In [201]:
'''
Dataframe with tls info
'''
df2 = df[df.tls.notnull()]

In [245]:
df2.index

Int64Index([  107,   108,   109,   110,   120,   121,   124,   134,   137,
              139,
            ...
            88834, 88837, 88841, 88844, 88861, 88868, 88892, 88913, 88914,
            88915],
           dtype='int64', length=8115)

In [ ]:
'''
Data Features:

Netflow:
1. Source Address - sa
2. Destination Address - da
3. Protocol - pr
4. Source Protocol -sp
5. Destination Protocol -dp

1. Features based on observable metadata, such as SPLTS and inter-arrival times, were used, and modeled as Markov chains.
2. Exclude TCP re-transmission by  tracking TCP sequence number
3. Assume packet lenths to be size of UDP, TCP, or ICMP packet payloads.
4. If the packet is neither UDP, TCP or ICMP, the length of the packet is set to the size of the IP packet
5. Inter-arrival times has a milliseconf resolution.
6. For length and times (SPLT), values are discreitzed into equally sized bins.
7. The length data Markov chain has 10 bins of 150 bytes each.
8. A 1500 byte MTU was assumed, and any packets observed with a size greater than 1350 bytes were put into the same bin.
9. The timing data Markov chain used 50 ms bins and 10 bins for 100 total features.
10. Inter-packet time greater than 450ms falls into the same bin.
*11. Transition probabilities used as features for algorithm.

Byte Distribution
12. Represent byte distribution as a 256-length array that keeps a count for each byte value encountered in the payloads of the packets of the flow being analyzed.
13. The byte value probabilities of a flow can be easily computed given the byte distribution by dividing the byte distribution counts by the total number of bytes found in the packet payloads.
14. The feature used by the ML algo are the 256 byte distribution probabilities

TLS Data
Client-based features
    1. List of offered ciphersuites (cs) - created a binary vector of length 176 where a one is assigned to each hexcode in the sameple's list of offered ciphersuites.
    2. List of advertised extensions - created a binary vector of length 21 to represent the TLS extensions.
    3. client's public key length

Server-based features
    1. List of selected ciphersuites
    2. Supported extensions
    3. Number of certificates
    4. Number of SAN names
    5. Validity in days
    6. Flag to check whether any of the certificates were self-signed.
    
NOTE: 176 unique hex codes advertised cs
      21 TLS extensions

DNS data
From the associated DNS response of the TLS flow, we collect:
1. Length of the domain name
2. Length of the FQDN
3. Binary feature for each of the 40 most common suffix and "other".
4. Binary feature for list of the 32 most common TTL values and an "other" option.
5. No  of numerical characters
6. No of non-alphanumeric characters
7. No. of IP addresses returned by the DNS response
8. 6 binary features flagging whether the domain name was in top-100, top-1000, toop-10000,top-100000, top-1000000 in the Alexa list

NOTE: For 8., the most prestigious category was chosen for each domain name i.e. top-100 = 1, mot the top-1000 feature

HTTP Data
1. Collect all HTTP flows from the same sa within a 5minute window of the TLS flow. 
   Represent all of the observed headers with a single feature vector of binary variables.
   If any of the HTTP flows have a specific header value, then that feature will be a 1 regardless of the other HTTP flows.
* 2. Use seven types of features from HTTP data. For each feature, select:
    - Presence of inbound and 
    - outbound HTTP fields
    - Content-Type
    - User-Agent
    - Accept-Language
    - Server
    - Code
'''

### Preprocessing

In [388]:
ps = contents[100:130]

In [214]:
#ciphersuite
cs = {u'c005': 0, u'c004': 1, u'c007': 2, u'0039': 3, u'0038': 4, u'c003': 5, u'c002': 6, u'0035': 7, u'0034': 8, u'0037': 9, u'0036': 10, u'c009': 11, u'c008': 12, u'0033': 13, u'0032': 14, u'c07a': 15, u'c07b': 16, u'c07c': 17, u'c07d': 18, u'0065': 19, u'c087': 21, u'c086': 22, u'c081': 23, u'c080': 24, u'c072': 25, u'c073': 26, u'c076': 27, u'c077': 28, u'0040': 29, u'0041': 30, u'0042': 31, u'feff': 32, u'0044': 33, u'0045': 34, u'0046': 35, u'0030': 36, u'c00e': 37, u'c00d': 38, u'c00f': 39, u'c00a': 40, u'c00c': 41, u'003e': 42, u'003d': 43, u'003f': 44, u'003a': 45, u'003c': 46, u'003b': 47, u'006a': 109, u'00ff': 49, u'00fd': 50, u'00fb': 51, u'00fc': 52, u'c08a': 53, u'c08b': 54, u'5600': 55, u'c05d': 56, u'c05c': 57, u'00af': 58, u'00ae': 59, u'0017': 113, u'00a7': 60, u'00a6': 61, u'00a5': 62, u'00a4': 63, u'00a3': 64, u'00a2': 65, u'00a1': 66, u'00a0': 67, u'cc13': 68, u'cc15': 69, u'cc14': 70, u'c049': 71, u'c048': 20, u'0016': 114, u'000d': 74, u'000f': 75, u'000a': 76, u'000c': 77, u'0064': 81, u'0066': 79, u'0067': 80, u'00b0': 78, u'00b1': 82, u'0062': 83, u'0063': 84, u'0060': 85, u'0061': 86, u'0068': 87, u'0069': 88, u'0004': 89, u'0005': 90, u'0006': 91, u'0007': 92, u'0001': 93, u'0002': 94, u'0003': 95, u'0008': 96, u'0009': 97, u'0031': 98, u'0019': 125, u'0018': 126, u'c030': 101, u'c031': 102, u'c032': 103, u'006d': 104, u'00ba': 105, u'006b': 106, u'006c': 107, u'00bd': 108, u'00be': 48, u'00c4': 110, u'00c0': 111, u'00c3': 112, u'cca9': 72, u'cca8': 73, u'0015': 115, u'0014': 116, u'0013': 117, u'0012': 118, u'0011': 119, u'0010': 120, u'c01b': 121, u'c01c': 122, u'c01a': 123, u'c01f': 124, u'c01d': 99, u'c01e': 100, u'002c': 127, u'002f': 128, u'c029': 129, u'c028': 130, u'c027': 131, u'c026': 132, u'c025': 133, u'c024': 134, u'c023': 135, u'c022': 136, u'c021': 137, u'c020': 138, u'009f': 139, u'009e': 140, u'009d': 141, u'009c': 142, u'009b': 143, u'009a': 144, u'0088': 145, u'0089': 146, u'0084': 147, u'0085': 148, u'0086': 149, u'0087': 150, u'0043': 151, u'c02f': 152, u'c02e': 153, u'c02d': 154, u'c02c': 155, u'c02b': 156, u'c02a': 157, u'c018': 158, u'c019': 159, u'001b': 160, u'001a': 161, u'c012': 162, u'c013': 163, u'c011': 164, u'c016': 165, u'c017': 166, u'c014': 167, u'008d': 168, u'008a': 169, u'008b': 170, u'008c': 171, u'0099': 172, u'0098': 173, u'0097': 174, u'0096': 175}

#extensions
ext = {u'server_name': 0, u'extended_master_secret': 1, u'renegotiation_info': 2, u'supported_groups': 3, u'ec_point_formats': 4, u'session_ticket': 5, u'application_layer_protocol_negotiation': 6, u'status_request': 7, u'signature_algorithms': 8}

In [383]:
def getTLSInfo(contents):
    #Add if contents == []: return None
    data = []
    for content in contents:
        if len(content["packets"]) == 0:
            continue
        tls_info = np.zeros(len(cs.keys()) + len(ext.keys()) + 1)

        if 'tls' in content and 'cs' in content['tls']:
            for c in content['tls']['cs']:
                if c in cs:
                    tls_info[cs[c]] = 1
        else:
            data.append([])
            continue

        if 'tls' in content and 'c_extensions' in content['tls']:
            for e in content['tls']['c_extensions']:
                if list(e)[0] in ext:
                    tls_info[len(cs.keys()) + ext[list(e)[0]]] = 1

        if 'tls' in content and 'c_key_length' in content['tls']:
            tls_info[len(cs.keys()) + len(ext.keys())] = content['tls']['c_key_length']
            
        data.append(list(tls_info))
    return data

In [389]:
data = getTLSInfo(ps)

In [392]:
def getByteDistribution(contents):
    #Add if contents == []: return None
    data = []
    for content in contents:
        if len(content['packets']) == 0:
            continue
        if 'byte_dist' in content and np.sum(content['byte_dist']) > 0:
            tmp = list(map(lambda x: np.divide(x,float(np.sum(content['byte_dist']))) , content['byte_dist']))
            data.append(tmp)
        else:
            data.append(np.zeros(256))
    return data

In [394]:
data = getByteDistribution(ps)

In [395]:
def getIndividualFlowPcktLengths(contents, nrows = 10, bin_size = 150.0):
    #Try nrow = 60, binsize = 25.0
    #Add if contents == []: return None
    '''
    Input: List of dictionary
    Example: [{'b': 151, 'dir': '>', 'ipt': 23},
             {'b': 1460, 'dir': '<', 'ipt': 21},
             {'b': 1286, 'dir': '<', 'ipt': 0}, 
             {'b': 150, 'dir': '>', 'ipt': 14}]
    '''
    
    data = []
    
    for content in contents:
        tx_matrix = np.zeros((nrows, nrows))
        if len(content['packets']) == 0:
            continue
        elif len(content['packets']) == 1:
            current_pkt_size = min(int(content['packets'][0]['b']/bin_size), nrows-1)
            tx_matrix[current_pkt_size, current_pkt_size] = 1
            data.append(list(tx_matrix.flatten()))
            continue
        
        #get raw transition count
        for i in range(1, len(content['packets'])):
            previous_pkt_size = min(int(content['packets'][i-1]['b']/bin_size), nrows-1)
            if 'b' not in content['packets'][i]:
                break
            current_pkt_size = min(int(content['packets'][i]['b']/bin_size), nrows-1)
            tx_matrix[previous_pkt_size, current_pkt_size] +=1
            
        #get empirical transition probabilities
        for i in range(nrows):
            if float(np.sum(tx_matrix[i:i+1])) != 0:
                tx_matrix[i:i+1] = tx_matrix[i:i+1]/float(np.sum(tx_matrix[i:i+1]))

        data.append(list(tx_matrix.flatten()))
        
    return data

In [397]:
data = getIndividualFlowPcktLengths(ps)

In [398]:
def getIndividualFlowIPTs(contents, nrows = 10, bin_size =50.0):
    #Try nrows = 30, bin_size = 50.0
    
    data = []
    for content in contents:
        tx_matrix = np.zeros((nrows,nrows))
        if len(content['packets']) == 0:
            continue
        elif len(content['packets']) == 1:
            current_IPT = min(int(content['packets'][0]['ipt']/float(bin_size)), nrows-1)
            tx_matrix[current_IPT, current_IPT] = 1
            data.append(list(tx_matrix.flatten()))
            continue
        
        # get raw transition counts
        for i in range(1, len(content['packets'])):
            previous_IPT = min(int(content['packets'][i-1]['ipt']/float(bin_size)), nrows-1)
            current_IPT = min(int(content['packets'][i]['ipt']/float(bin_size)), nrows-1)
            tx_matrix[previous_IPT, current_IPT]
            
        # get empirical transition probabilities
        for i in range(nrows):
            if float(np.sum(tx_matrix[i:i+1])) != 0:
                tx_matrix[i:i+1] = tx_matrix[i:i+1]/float(np.sum(tx_matrix[i:i+1]))
                
        data.append(list(tx_matrix.flatten()))
        
    return data

In [399]:
data = getIndividualFlowIPTs(ps)

26

In [381]:
def getIndividualFlowMetadata(contents):
    data = []
    for content in contents:
        if len(content['packets']) == 0:
            continue
        tmp = []
        
        key = content['sa'].replace('.',',') +\
              content['da'].replace('.',',')+\
              str(content['sp'])+\
              str(content['dp'])+\
              str(content['pr'])
     
        
        if content['dp'] != None:
            tmp.append(float(content['dp'])) # destination port
        else:
            tmp.append(0) #ICMP/etc.
        if content['sp'] != None:
            tmp.append(float(content['sp'])) # source port
        else:
            tmp.append(0) #ICMP/etc.      
        if 'num_pkts_in' in content:
            tmp.append(content['num_pkts_in']) # inbound packets
        else:
            tmp.append(0)
        if 'num_pkts_out' in content:
            tmp.append(content['num_pkts_out']) # outbound packets
        else:
            tmp.append(0)
        if 'bytes_in' in content:
            tmp.append(content['bytes_in']) # inbound bytes
        else:
            tmp.append(0)
        if 'bytes_out' in content:
            tmp.append(content['bytes_out']) # outbound bytes
        else:
            tmp.append(0)
        if content['packets'] == []: # elapsed time of flow
            tmp.append(0)
        else:
            time = 0
            for packet in content['packets']:
                time+=packet['ipt']
            tmp.append(time)
            
        data.append(tmp)
        
    if data == []:
        return None
    return data 

In [400]:
getIndividualFlowMetadata(ps)

[[80.0, 49206.0, 4, 5, 1898, 727, 35],
 [55481.0, 53.0, 1, 1, 32, 48, 4294967295],
 [80.0, 49207.0, 5, 5, 3491, 832, 45],
 [80.0, 49208.0, 7, 6, 6661, 796, 40],
 [137.0, 137.0, 0, 7, 0, 350, 42278],
 [54444.0, 53.0, 1, 1, 27, 43, 4294967295],
 [443.0, 49209.0, 9, 10, 4863, 770, 120],
 [443.0, 49210.0, 6, 7, 2821, 301, 72],
 [443.0, 49211.0, 10, 9, 6049, 1098, 113],
 [443.0, 49212.0, 12, 9, 9275, 1062, 95],
 [60473.0, 53.0, 1, 1, 35, 51, 4294967295],
 [80.0, 49214.0, 98, 155, 44626, 1587, 3947],
 [137.0, 137.0, 0, 2, 0, 100, 763],
 [49194.0, 445.0, 1, 1, 0, 1, 0],
 [59481.0, 53.0, 1, 1, 39, 55, 4294967295],
 [59482.0, 53.0, 1, 1, 45, 80, 4294967295],
 [59483.0, 53.0, 1, 1, 47, 118, 4294967295],
 [59485.0, 53.0, 1, 1, 34, 50, 4294967295],
 [443.0, 49215.0, 10, 9, 5606, 1025, 92],
 [443.0, 49216.0, 9, 9, 4943, 770, 141],
 [137.0, 137.0, 0, 3, 0, 150, 1523],
 [52157.0, 53.0, 1, 1, 36, 52, 4294967295],
 [443.0, 49217.0, 8, 9, 1281, 1275, 66935],
 [55183.0, 53.0, 1, 1, 48, 320, 4294967295],


### Parse Dataset

In [ ]:
'''
Load files
'''
path_to_json = "Dataset/json/test/"
contents = []
pattern = os.path.join(path_to_json, '*')
file_list = glob.glob(pattern)
c = 0

label = []
l = 0
for file in file_list:
    l_ = []
    try:
        with open(file, 'r', encoding='cp1252') as f:
            next(f)
            for line in f:
                try:
                    contents.append(json.loads(line))
                    l_.append(l)
                    
                except:
                    c+=1
                    pass
    except:
        print(f'Error: failed to parse file {file}') 
        pass
    l += 1

In [ ]:
def process_data():
    n_of_files = 0
    data = []
    label = []
    
    '''
    Store malicious and normal traffic in separate folders.
    
    Load using:
    if malicious_dir != None:
        load_data(malicious_dir, 1.0)
        
        
    '''

In [479]:
def load_data(data_dir):
    '''
    Check for TLS info before parsing
    '''
    
    '''
    Store malicious and normal traffic in separate folders.
    
    Load using:
    if malicious_dir != None:
        load_data(malicious_dir, 1.0)
        
        
    '''
    c  = 0
    data = []
#     size_check = []
    data_dir = os.path.join(data_dir, '*')
    files = glob.glob(data_dir)
    n_of_files = 0
    for file in files:
        contents = []
        try:
            with open(file, 'r', encoding='cp1252') as f:
                next(f)
                for line in f:
                    try:
                        contents.append(json.loads(line))
                    except:
                        print(f'Error: failed to parse file {file}')
                        c+=1
                        pass
        except:
            print(f'Error: failed to parse file {file}') 
            continue
        
        
        
        tmpTLS = getTLSInfo(contents)
        tmpBD = getByteDistribution(contents)
        tmpIPT = getIndividualFlowIPTs(contents)
        tmpPL = getIndividualFlowPcktLengths(contents)
        tmpMTD = getIndividualFlowMetadata(contents) 
        
        if tmpMTD != None and tmpPL != None and tmpIPT != None and tmpTLS != None:
            #if all([x!=0, y!=0, z!=0])
            for i in range(len(tmpMTD)):
                tmp_data = []
                size_tmp = []
                
                tmp_data.extend(tmpTLS[i])
                tmp_data.extend(tmpBD[i])
                tmp_data.extend(tmpIPT[i])
                tmp_data.extend(tmpPL[i])
                tmp_data.extend(tmpMTD[i])
                
#                 size_tmp.append(len(tmpTLS[i]))
#                 size_tmp.append(len(tmpBD[i]))
#                 size_tmp.append(len(tmpIPT[i]))
#                 size_tmp.append(len(tmpPL[i]))
#                 size_tmp.append(len(tmpMTD[i]))
                
                data.append(tmp_data)
        
        n_of_files+=1
    print("completed processing")
    print(f"{c} lines not read")
    return data

In [516]:
data_dir = 'Dataset/json/test/neg/'
data = load_data(data_dir)

completed processing
0 lines not read


In [518]:
neg = []
label = []
for i in range(len(data)):
    if len(data[i]) > 470:
        neg.append(data[i])

In [520]:
data_dir = 'Dataset/json/test/pos/'
data = load_data(data_dir)

completed processing
0 lines not read


In [521]:
pos = []
label = []
for i in range(len(data)):
    if len(data[i]) > 470:
        pos.append(data[i])

In [522]:
neg_label = [0] * len(neg)
pos_label = [1] * len(pos)

### Classifier

In [495]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [523]:
pos.extend(neg)
pos_label.extend(neg_label)

In [527]:
X = np.array(pos)

In [528]:
y = pos_label

In [540]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    random_state=42,
                                                    stratify=y)

In [541]:
# Standardize
scaler = StandardScaler() #Scale only training dataset
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [542]:
# Train
lr = LogisticRegression(penalty='l1')
lr.fit(X_train, y_train)

C:\Users\dubem\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [543]:
#Test
pred = lr.predict(X_test)

In [544]:
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

0.64
              precision    recall  f1-score   support

           0       0.62      0.67      0.64        12
           1       0.67      0.62      0.64        13

   micro avg       0.64      0.64      0.64        25
   macro avg       0.64      0.64      0.64        25
weighted avg       0.64      0.64      0.64        25

[[8 4]
 [5 8]]


In [549]:

from sklearn.linear_model import LassoCV


# cv_outer = KFold(len(X), n_folds=5)
lasso = LassoCV(cv=5, alphas=0.1)
lasso.fit(X_train, y_train)
pred = lasso.predict(X_test)

print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

AxisError: axis -1 is out of bounds for array of dimension 0

In [403]:
import subprocess
subprocess.call("sleep.sh", shell=True) #shell=True is bad for security. Add path/to/file instead


# res = subprocess.check_output(["sudo", "apt", "update"])
# for line in res.splitlines():

In [402]:
# k = []
# for p in ps:
#     if 'byte_dist' in p and sum(p['byte_dist']) > 0:
#         tmp = list(map(lambda x: x/float(sum(p['byte_dist'])) , p['byte_dist']))
#         k.append(tmp)
#     else:
#         k.append(np.zeros(256))
# k

In [401]:
# bd = ps[0]['byte_dist']
# list(map(lambda x: x/float(sum(bd)), bd))

In [313]:
tm

In [257]:
ps[0]['tls']['c_extensions']

[{'renegotiation_info': '00'},
 {'server_name': '000c000009676d61696c2e636f6d'},
 {'supported_groups': '000400170018'},
 {'ec_point_formats': '0100'},
 {'signature_algorithms': '000e0401050102010403050302030202'}]

In [ ]:
http":[{"in":[{"version":"HTTP/1.1"},
              {"code":"200"},
              {"reason":"OK"},
              {"Server":"nginx/1.6.2"},
              {"Date":"Wed, 08 Apr 2015 10:29:59 GMT"},
              {"Content-Type":"text/html"},
              {"Content-Length":"16"},
              {"Connection":"keep-alive"},
              {"body":"4735546f7a4f454d7333784976783937"}
             ]
     }],"probable_os":{"out":"Windows XP"},"expire_type":"i"}

In [420]:
# df[df.tls.notnull() & df.http.notnull()].info

In [96]:
pd.DataFrame(df[df.tls.notnull() & df.http.notnull()].info)

ValueError: DataFrame constructor not properly called!

In [46]:
path = "Dataset/json/snort_log1"
k = []
# with open(path) as f:
#     data = f.read()
#     content = json.loads(data)
for line in open(path, 'r'):
    k.append(line)
# pd.DataFrame(k)#"Dataset/json/snort_log1", lines=True)

In [48]:
js = json.loads(json.dumps(k))

In [50]:
pd.DataFrame

TypeError: __init__() got an unexpected keyword argument 'orient'

In [56]:
d = pd.read_csv("ASNM-CDX-2009.csv", sep=";", index_col="id")

FileNotFoundError: File b'ASNM-CDX-2009.csv' does not exist

In [9]:
d.shape

(5771, 876)

In [13]:
d.isnull().sum().sum()

0

In [12]:
for c in d.columns:
    print(c)

label_2
label_poly
SrcIP
DstIP
SrcPort
DstPort
SrcMAC
DstMAC
SrcIPInVlan
DstIPInVlan
InPkt1s10i[0]
InPkt1s10i[1]
InPkt1s10i[2]
InPkt1s10i[3]
InPkt1s10i[4]
InPkt1s10i[5]
InPkt1s10i[6]
InPkt1s10i[7]
InPkt1s10i[8]
InPkt1s10i[9]
InPkt4s10i[0]
InPkt4s10i[1]
InPkt4s10i[2]
InPkt4s10i[3]
InPkt4s10i[4]
InPkt4s10i[5]
InPkt4s10i[6]
InPkt4s10i[7]
InPkt4s10i[8]
InPkt4s10i[9]
InPkt8s10i[0]
InPkt8s10i[1]
InPkt8s10i[2]
InPkt8s10i[3]
InPkt8s10i[4]
InPkt8s10i[5]
InPkt8s10i[6]
InPkt8s10i[7]
InPkt8s10i[8]
InPkt8s10i[9]
InPkt32s10i[0]
InPkt32s10i[1]
InPkt32s10i[2]
InPkt32s10i[3]
InPkt32s10i[4]
InPkt32s10i[5]
InPkt32s10i[6]
InPkt32s10i[7]
InPkt32s10i[8]
InPkt32s10i[9]
InPkt64s10i[0]
InPkt64s10i[1]
InPkt64s10i[2]
InPkt64s10i[3]
InPkt64s10i[4]
InPkt64s10i[5]
InPkt64s10i[6]
InPkt64s10i[7]
InPkt64s10i[8]
InPkt64s10i[9]
OutPkt1s10i[0]
OutPkt1s10i[1]
OutPkt1s10i[2]
OutPkt1s10i[3]
OutPkt1s10i[4]
OutPkt1s10i[5]
OutPkt1s10i[6]
OutPkt1s10i[7]
OutPkt1s10i[8]
OutPkt1s10i[9]
OutPkt4s10i[0]
OutPkt4s10i[1]
OutPkt4s10i[2]


In [3]:
df1 = pd.read_json("fingerprint_db.json", lines=True)

In [5]:
df1.shape

(4978, 7)

In [6]:
df1.head(10)

,max_implementation_date,md5_repr,min_implementation_date,process_info,source,str_repr,tls_features
0,2015-09,6d0ceee6c09d46a1ae96ac6388fac84f,1999-01,"[{'process': 'OUTLOOK.EXE', 'application_categ...",[Cisco],(0303)(c02cc02bc030c02f009f009ec024c023c028c02...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
1,2018-10,a29cccb0905fae70c61b3cd4e3c6f340,1999-01,"[{'process': 'chrome.exe', 'application_catego...",[Cisco],(0303)(0a0a130113021303c02bc02fc02cc030cca9cca...,"{'version': 'TLS 1.2', 'cipher_suites': ['GREA..."
2,2012-02,03eb3f9a3ac84e0c449817f7d2e85dba,2003-06,"[{'process': 'Webex Teams', 'application_categ...",[Cisco],(0303)(c030c02cc02fc02bc028c024c027c023009d009...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
3,2015-09,d5c2bed29522e1118c8114b81d744dbb,1999-01,"[{'process': 'Microsoft Outlook', 'application...",[Cisco],(0303)(00ffc02cc02bc024c023c00ac009c008c030c02...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
4,2018-08,0e4fdd51d024b22074b16694b54e39d2,1999-01,"[{'process': 'firefox.exe', 'application_categ...",[Cisco],(0303)(130113031302c02bc02fcca9cca8c02cc030c00...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
5,2018-10,289c98a39674e74fe60e27ad26524b7a,1999-01,"[{'process': 'chrome.exe', 'application_catego...",[Cisco],(0303)(0a0a130113021303c02bc02fc02cc030cca9cca...,"{'version': 'TLS 1.2', 'cipher_suites': ['GREA..."
6,2018-08,5deb9017c3ce55b5b941378f70866c41,1999-01,"[{'process': 'Microsoft PowerPoint', 'applicat...",[Cisco],(0303)(130313011302c02cc02bc024c023c00ac009cca...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
7,2018-10,6ba250cea6f9b71af1f5c4e0be0be651,1999-01,"[{'process': 'MicrosoftEdgeCP.exe', 'applicati...",[Cisco],(0303)(c02cc02bc030c02fc024c023c028c027c00ac00...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
8,2015-10,84a23cb5d3e1cf0af3cc8d1ab0b89a15,1999-01,"[{'process': 'ampdaemon', 'application_categor...",[Cisco],(0303)(c030c02cc028c024c014c00a00a500a300a1009...,"{'version': 'TLS 1.2', 'cipher_suites': ['TLS_..."
9,2018-10,8f631441c725eeaf302dcbca4602c976,1999-01,"[{'process': 'chrome.exe', 'application_catego...",[Cisco],(0303)(0a0a130113021303c02bc02fc02cc030cca9cca...,"{'version': 'TLS 1.2', 'cipher_suites': ['GREA..."


In [13]:
df1.process_info[1]

[{'application_category': 'browser',
  'os_info': {'(WinNT)(Windows 10 Enterprise)(10.0.17134)': 0.93,
   '(WinNT)(Windows 10 Enterprise)(10.0.17763)': 0.02,
   '(WinNT)(Windows 10 Enterprise)(10.0.18362)': 0.02},
  'prevalence': 0.30000000000000004,
  'process': 'chrome.exe',
  'sha256': '2AD6823E4ED4920385A098F4510975B28C24DF80F1A22F677A6067D84657A5F1'},
 {'application_category': 'browser',
  'os_info': {'(Mac OS X)(Mojave)(10.14.6)': 0.2,
   '(Mac OS X)(Unknown)(10.14.5)': 0.19,
   '(Mac OS X)(Unknown)(10.14.6)': 0.17},
  'prevalence': 0.24,
  'process': 'Google Chrome Helper',
  'sha256': 'CC0A1376F3DD7B696BA0BA06E59598F7A22305985AEA39B999ABF19252E20EB2'},
 {'application_category': 'browser',
  'os_info': {'(WinNT)(Windows 10 Enterprise)(10.0.17134)': 0.91,
   '(WinNT)(Windows 10 Enterprise)(10.0.17763)': 0.02,
   '(WinNT)(Windows 7 Enterprise)(6.1.7601)': 0.02},
  'prevalence': 0.11,
  'process': 'chrome.exe',
  'sha256': 'BE3D2BA94E34BAC4EC82540FB9C849E7E1B320359F98A1C3AFAC5A9EBD

In [14]:
df1.str_repr[0]

'(0303)(c02cc02bc030c02f009f009ec024c023c028c027c00ac009c014c013009d009c003d003c0035002f000a)((0000)(000500050100000000)(000a00080006001d00170018)(000b00020100)(000d00140012040105010201040305030203020206010603)(0023)(0017)(ff01))'

In [16]:
df1.tls_features[1]

{'cipher_suites': ['GREASE',
  'TLS_AES_128_GCM_SHA256',
  'TLS_AES_256_GCM_SHA384',
  'TLS_CHACHA20_POLY1305_SHA256',
  'TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256',
  'TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256',
  'TLS_ECDHE_ECDSA_WITH_AES_256_GCM_SHA384',
  'TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384',
  'TLS_ECDHE_ECDSA_WITH_CHACHA20_POLY1305_SHA256',
  'TLS_ECDHE_RSA_WITH_CHACHA20_POLY1305_SHA256',
  'TLS_ECDHE_RSA_WITH_AES_128_CBC_SHA',
  'TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA',
  'TLS_RSA_WITH_AES_128_GCM_SHA256',
  'TLS_RSA_WITH_AES_256_GCM_SHA384',
  'TLS_RSA_WITH_AES_128_CBC_SHA',
  'TLS_RSA_WITH_AES_256_CBC_SHA',
  'TLS_RSA_WITH_3DES_EDE_CBC_SHA'],
 'extensions': [{'GREASE': ''},
  {'server_name': ''},
  {'extended_master_secret': ''},
  {'renegotiation_info': ''},
  {'supported_groups': {'supported_groups': ['GREASE',
     'x25519',
     'secp256r1',
     'secp384r1'],
    'supported_groups_list_length': 8}},
  {'ec_point_formats': {'ec_point_formats': ['uncompressed'],
    'ec_point_form

In [2]:
df = pd.read_csv("packetcap.csv")

In [9]:
df.head(20)

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,192.168.1.137,192.168.3.255,BROWSER,254,"Host Announcement JACLYNS-15, Workstation, Ser..."
1,2,0.000765,192.168.1.54,192.168.3.255,NBNS,92,Name query NB GMS-UX331UA<20>
2,3,0.203739,192.168.1.255,192.168.3.255,UDP,305,54915 > 54915 Len=263
3,4,0.246295,192.168.1.246,52.114.77.34,TCP,54,"49695 > 443 [FIN, ACK] Seq=1 Ack=1 Win=1020 ..."
4,5,0.321706,192.168.0.214,239.255.255.250,SSDP,217,M-SEARCH * HTTP/1.1
5,6,0.350479,192.168.1.145,239.255.255.250,SSDP,215,M-SEARCH * HTTP/1.1
6,7,0.389751,52.114.77.34,192.168.1.246,TCP,60,"443 > 49695 [FIN, ACK] Seq=1 Ack=2 Win=1026 ..."
7,8,0.389814,192.168.1.246,52.114.77.34,TCP,54,[TCP Dup ACK 4#1] 49695 > 443 [ACK] Seq=2 Ac...
8,9,0.389916,192.168.1.246,52.114.77.34,TCP,54,49695 > 443 [ACK] Seq=2 Ack=2 Win=1020 Len=0
9,10,0.560849,192.168.2.98,239.255.255.250,SSDP,217,M-SEARCH * HTTP/1.1
